In [2]:
pip install librosa==0.8.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.8/203.8 kB 3.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 43.1 MB/s eta 0:00:0000:0100:01
^C
ERROR: Operation cancelled by user
Note: you may need to restart the kernel to use updated packages.


In [1]:
import torch
import numpy as np
import librosa

## Call model
Forked from https://github.com/RF5/simple-speaker-embedding

In [4]:
model = torch.hub.load('EmilYJuice/simple-speaker-embedding', 'convgru_embedder')
model.eval()

/opt/conda/lib/python3.10/site-packages/torch/hub.py:294: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/EmilYJuice/simple-speaker-embedding/zipball/master" to /root/.cache/torch/hub/master.zip
Downloading: "https://github.com/RF5/simple-speaker-embedding/releases/download/v1.0/convgru_ckpt_00700000_strip.pt" to /root/.cache/torch/hub/checkpoints/convgru_ckpt_00700000_strip.pt
100%|██████████| 121M/121M [00:00<00:00, 185MB

ConvGRUEmbedder(
  (model): ConvRNNEmbedder(
    (conv_encoder): ConvEncoder(
      (conv_layers): ModuleList(
        (0): Sequential(
          (0): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
          (1): Dropout(p=0.0, inplace=False)
          (2): GroupNorm(512, 512, eps=1e-05, affine=True)
          (3): GELU(approximate='none')
        )
        (1-4): 4 x Sequential(
          (0): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (1): Dropout(p=0.0, inplace=False)
          (2): GELU(approximate='none')
        )
        (5-6): 2 x Sequential(
          (0): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
          (1): Dropout(p=0.0, inplace=False)
          (2): GELU(approximate='none')
        )
      )
    )
    (rnn): GRU(512, 768, num_layers=3, batch_first=True, dropout=0.3, bidirectional=True)
    (head): Linear(in_features=1536, out_features=256, bias=True)
  )
)

In [3]:
import shutil
import os

## Prepare dataset (first 40 speakers from VCTK)

In [6]:
# extract p225,p226,p227,p228
source_folder = '/kaggle/input/vctk48'
output_folder = '/kaggle/working/new_wavs'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

In [7]:
all_folders = [os.path.join(source_folder, f) for f in os.listdir(source_folder) if os.path.isdir(os.path.join(source_folder, f))]

In [8]:
all_folders.sort()
selected_folders = all_folders[:40]

In [9]:
for folder in selected_folders:
    # Define the destination path for the folder
    dest_path = os.path.join(output_folder, os.path.basename(folder))
    
    # Copy the folder to the destination
    shutil.copytree(folder, dest_path)

## Downsampling all wav file to 16k Hz

In [11]:
import soundfile as sf

In [12]:
# downsampling for all wav files
rootDir = '/kaggle/working/new_wavs'

dirName, subdirList, _ = next(os.walk(rootDir))

for subdir in sorted(subdirList):
    _,_, fileList = next(os.walk(os.path.join(dirName,subdir)))
    for fileName in sorted(fileList):
        filepath = os.path.join(rootDir, subdir, fileName)
        audio, sample_rate = librosa.load(filepath, sr=16000)
        output_file_path = filepath  
        sf.write(output_file_path, audio, sample_rate)

## generate train.pkl (speaker embedding "convgru")

In [13]:
# Directory containing mel-spectrograms
rootDir = '/kaggle/input/679-40-wavs/new_spmel'
dirName, subdirList, _ = next(os.walk(rootDir))
print('Found directory: %s' % dirName)

Found directory: /kaggle/input/679-40-wavs/new_spmel


In [14]:
import pickle

In [15]:
speakers = []
for speaker in sorted(subdirList):
    print('Processing speaker: %s' % speaker)
    utterances = []
    utterances.append(speaker)
    _, _, fileList = next(os.walk(os.path.join(dirName,speaker)))
    
    # make speaker embedding
    assert len(fileList) >= num_uttrs
    idx_uttrs = np.random.choice(len(fileList), size=num_uttrs, replace=False)
    embs = []
    for i in range(num_uttrs):
        tmp = np.load(os.path.join(dirName, speaker, fileList[idx_uttrs[i]]))
        candidates = np.delete(np.arange(len(fileList)), idx_uttrs)
        # choose another utterance if the current one is too short
        while tmp.shape[0] < len_crop:
            idx_alt = np.random.choice(candidates)
            tmp = np.load(os.path.join(dirName, speaker, fileList[idx_alt]))
            candidates = np.delete(candidates, np.argwhere(candidates==idx_alt))
        left = np.random.randint(0, tmp.shape[0]-len_crop)
        melsp = torch.from_numpy(tmp[np.newaxis, left:left+len_crop, :]).cuda()
        emb = C(melsp)
        embs.append(emb.detach().squeeze().cpu().numpy())     
    utterances.append(np.mean(embs, axis=0))
    
    # create file list
    for fileName in sorted(fileList):
        utterances.append(os.path.join(speaker,fileName))
    speakers.append(utterances)
    
with open(os.path.join('/kaggle/working/', 'train_40_wavs.pkl'), 'wb') as handle:
    pickle.dump(speakers, handle)

Processing speaker: p225
Processing speaker: p228
Processing speaker: p233
Processing speaker: p256
Processing speaker: p257
Processing speaker: p258
Processing speaker: p266


In [7]:
# generate speaker embedding
speakers = []
# Assuming subdirList is a list of speaker directory names under dirName
for speaker in sorted(subdirList):
    print('Processing speaker: %s' % speaker)
    utterances = [speaker]  # Start with speaker name
    
    speaker_dir = os.path.join(dirName, speaker)
    _, _, fileList = next(os.walk(speaker_dir))
    
    embs = []
    for i in fileList:
#         mel = model.melspec_from_file(os.path.join(speaker_dir, i))
#         embedding = model(mel[None])
        wav, _ = librosa.load(os.path.join(speaker_dir, i), sr=16000)
        wav = torch.from_numpy(wav).float()
        embedding = model(wav[None])
        embs.append(embedding.detach().squeeze().cpu().numpy())     
    utterances.append(np.mean(embs, axis=0))
    
    # create file list
    for fileName in sorted(fileList):
        utterances.append(os.path.join(speaker,fileName))
    speakers.append(utterances)

Processing speaker: p225


/tmp/ipykernel_34/1321384876.py:15: UserWarning: PySoundFile failed. Trying audioread instead.
  wav, _ = librosa.load(os.path.join(speaker_dir, i), sr=16000)
/opt/conda/lib/python3.10/site-packages/librosa/core/audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


NoBackendError: 

In [17]:
with open(os.path.join("/kaggle/working/", 'train_new_40.pkl'), 'wb') as handle:
    pickle.dump(speakers, handle)

## get_spectorm

In [ ]:
import os
import pickle
import numpy as np
import soundfile as sf
from scipy import signal
from scipy.signal import get_window
from librosa.filters import mel
from numpy.random import RandomState

In [ ]:
def butter_highpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = signal.butter(order, normal_cutoff, btype='high', analog=False)
    return b, a
    
    
def pySTFT(x, fft_length=1024, hop_length=256):
    
    x = np.pad(x, int(fft_length//2), mode='reflect')
    
    noverlap = fft_length - hop_length
    shape = x.shape[:-1]+((x.shape[-1]-noverlap)//hop_length, fft_length)
    strides = x.strides[:-1]+(hop_length*x.strides[-1], x.strides[-1])
    result = np.lib.stride_tricks.as_strided(x, shape=shape,
                                             strides=strides)
    
    fft_window = get_window('hann', fft_length, fftbins=True)
    result = np.fft.rfft(fft_window * result, n=fft_length).T
    
    return np.abs(result)    
    

In [ ]:
mel_basis = mel(sr=16000, n_fft=1024, fmin=90, fmax=7600, n_mels=80).T
min_level = np.exp(-100 / 20 * np.log(10))
b, a = butter_highpass(30, 16000, order=5)

In [ ]:
# audio file directory
rootDir = '/kaggle/working/new_wavs'
# spectrogram directory
targetDir = '/kaggle/working/new_spmel'


dirName, subdirList, _ = next(os.walk(rootDir))
print('Found directory: %s' % dirName)

for subdir in sorted(subdirList):
    print(subdir)
    if not os.path.exists(os.path.join(targetDir, subdir)):
        os.makedirs(os.path.join(targetDir, subdir))
    _,_, fileList = next(os.walk(os.path.join(dirName,subdir)))
    prng = RandomState(int(subdir[1:])) 
    for fileName in sorted(fileList):
        # Read audio file
        x, fs = sf.read(os.path.join(dirName,subdir,fileName))
        # Remove drifting noise
        y = signal.filtfilt(b, a, x)
        # Ddd a little random noise for model roubstness
        wav = y * 0.96 + (prng.rand(y.shape[0])-0.5)*1e-06
        # Compute spect
        D = pySTFT(wav).T
        # Convert to mel and normalize
        D_mel = np.dot(D, mel_basis)
        D_db = 20 * np.log10(np.maximum(min_level, D_mel)) - 16
        S = np.clip((D_db + 100) / 100, 0, 1)    
        # save spect    
        np.save(os.path.join(targetDir, subdir, fileName[:-4]),
                S.astype(np.float32), allow_pickle=False)    

In [ ]:
!zip -r file.zip /kaggle/working

In [ ]:
from IPython.display import FileLink
FileLink(r'file.zip')